In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [3]:
inputT1 = '/Shared/sinapse/CACHE/20160610_HCP_base_Results/HCP_DATA/105115/HCP_105115_01/TissueClassify/t1_average_BRAINSABC.nii.gz'
inputT2 = '/Shared/sinapse/CACHE/20160610_HCP_base_Results/HCP_DATA/105115/HCP_105115_01/TissueClassify/t2_average_BRAINSABC.nii.gz'
LobesLabelMapVolume = '/Shared/sinapse/CACHE/20160610_HCP_base_Results/HCP_DATA/105115/HCP_105115_01/JointFusion/JointFusion_HDAtlas20_2015_lobe_labe
DWI_corrected_alignedSpace_masked = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/DWI_corrected_alignedSpace_masked.nrrd'
DWI_baseline = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/DWI_Baseline.nrrd'
DWI_sr_nn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/DWI_SR_NN.nrrd'
DWI_sr_ifft = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/DWI_SR_IFFT.nrrd'
DWI_brainMask = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/DWIBrainMask.nrrd'

In [4]:
sessionID = 105115
WFname = 'HCPWorkflow_Dist_' + str(sessionID)

In [16]:
###### UTILITY FUNCTIONS #######
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/#
# 
def ComputeDistanceImages(DWI_baseline,DWI_sr,DWI_brainMask):
    import os
    import numpy as np
    from scipy.linalg import logm
    from scipy.linalg import eigvalsh # eigvalsh(A,B) is joint eigenvalues of A and B
    import SimpleITK as sitk
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
    #########
    ### distance functions ###
    # Frobenius distance
    def distance_euclid(tenfit_A, tenfit_B):
        return np.linalg.norm(tenfit_A.quadratic_form - tenfit_B.quadratic_form, ord='fro')
    # Log-Euclidian distance
    def distance_logeuclid(tenfit_A, tenfit_B):
        def fro_norm(A,B):
            return np.linalg.norm(A-B, ord='fro')
        return fro_norm(logm(tenfit_A.quadratic_form),logm(tenfit_B.quadratic_form))
    # Reimannian distance
    def distance_reimann(tenfit_A, tenfit_B):
        return np.sqrt((np.log(eigvalsh(tenfit_A.quadratic_form,tenfit_B.quadratic_form))**2).sum())
    # Kullback-Leibler distance
    def distance_kullback(tenfit_A, tenfit_B):
        A = tenfit_A.quadratic_form
        B = tenfit_B.quadratic_form
        dim = A.shape[0]
        kl = np.sqrt( np.trace( np.dot(np.linalg.inv(A),B)+np.dot(np.linalg.inv(B),A) ) - 2*dim )
        return 0.5*kl
    ######### 
    assert os.path.exists(DWI_baseline), "File not found: %s" % DWI_baseline
    assert os.path.exists(DWI_sr), "File not found: %s" % DWI_sr
    assert os.path.exists(DWI_brainMask), "File not found: %s" % DWI_brainMask
    mask = sitk.ReadImage(DWI_brainMask)
    # read DWI nrrd files
    data_base,nrrd_header_base,bvecs_base,bvals_base,grad_idx_base = ReadNAMICDWIFromNrrd(DWI_baseline)
    data_sr,nrrd_header_sr,bvecs_sr,bvals_sr,grad_idx_sr = ReadNAMICDWIFromNrrd(DWI_sr)
    # compute tensorfits
    from dipy.core.gradients import gradient_table
    from dipy.reconst.dti import TensorModel
    # base tensorfit
    gtab_base = gradient_table(bvals_base, bvecs_base)
    ten_base = TensorModel(gtab_base)
    tenfit_base = ten_base.fit(data_base)
    # super-res reconstructed tensorfit
    gtab_sr = gradient_table(bvals_sr, bvecs_sr)
    ten_sr = TensorModel(gtab_sr)
    tenfit_sr = ten_base.fit(data_sr)
    # create empty distance images
    size = [data_base.shape[0],data_base.shape[1],data_base.shape[2]] #data_base.shape[4] is gradient components
    fa_distance_image = sitk.Image(size, sitk.sitkFloat32)
    fa_distance_image.CopyInformation(mask)
    #
    md_distance_image = sitk.Image(size, sitk.sitkFloat32)
    md_distance_image.CopyInformation(mask)
    #
    rd_distance_image = sitk.Image(size, sitk.sitkFloat32)
    rd_distance_image.CopyInformation(mask)
    #
    ad_distance_image = sitk.Image(size, sitk.sitkFloat32)
    ad_distance_image.CopyInformation(mask)
    #
    frobenius_distance_image = sitk.Image(size, sitk.sitkFloat32)
    frobenius_distance_image.CopyInformation(mask)
    #
    logeuclid_distance_image = sitk.Image(size, sitk.sitkFloat32)
    logeuclid_distance_image.CopyInformation(mask)
    #
    reimann_distance_image = sitk.Image(size, sitk.sitkFloat32)
    reimann_distance_image.CopyInformation(mask)
    #
    kullback_distance_image = sitk.Image(size, sitk.sitkFloat32)
    kullback_distance_image.CopyInformation(mask)
    #
    # for loop to fill the distance images
    '''
    for i in xrange(size[0]):
        for j in xrange(size[1]):
            for k in xrange(size[2]):
                fa_distance_image[i,j,k] = abs(tenfit_base[i,j,k].fa - tenfit_sr[i,j,k].fa)
                md_distance_image[i,j,k] = abs(tenfit_base[i,j,k].md - tenfit_sr[i,j,k].md)
                rd_distance_image[i,j,k] = abs(tenfit_base[i,j,k].rd - tenfit_sr[i,j,k].rd)
                ad_distance_image[i,j,k] = abs(tenfit_base[i,j,k].ad - tenfit_sr[i,j,k].ad)
                frobenius_distance_image[i,j,k] = distance_euclid(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                logeuclid_distance_image[i,j,k] = distance_logeuclid(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                reimann_distance_image[i,j,k] = distance_reimann(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                kullback_distance_image[i,j,k] = distance_kullback(tenfit_base[i,j,k], tenfit_sr[i,j,k])
    '''
    #
    maskf = sitk.Cast(mask, sitk.sitkFloat32)
    #
    ## create proper SR prefix for output file name
    sr_file_name = os.path.basename(DWI_sr)
    sr_file_name_base = os.path.splitext(sr_file_name)[0]
    sr_name = sr_file_name_base.split('_',2)[2]
    #
    fa_distance_image_fn = os.path.realpath(sr_name + '_FA_distance.nrrd')
    md_distance_image_fn = os.path.realpath(sr_name + '_MD_distance.nrrd')
    rd_distance_image_fn = os.path.realpath(sr_name + '_RD_distance.nrrd')
    ad_distance_image_fn = os.path.realpath(sr_name + '_AD_distance.nrrd')
    frobenius_distance_image_fn = os.path.realpath(sr_name + '_Frobenius_distance.nrrd')
    logeuclid_distance_image_fn = os.path.realpath(sr_name + '_Logeuclid_distance.nrrd')
    reimann_distance_image_fn = os.path.realpath(sr_name + '_Reimann_distance.nrrd')
    kullback_distance_image_fn = os.path.realpath(sr_name + '_Kullback_distance.nrrd')
    #
    ## mask out the background
    fa_distance_image = sitk.Multiply(fa_distance_image,maskf)
    md_distance_image = sitk.Multiply(md_distance_image,maskf)
    rd_distance_image = sitk.Multiply(rd_distance_image,maskf)
    ad_distance_image = sitk.Multiply(ad_distance_image,maskf)
    frobenius_distance_image = sitk.Multiply(frobenius_distance_image,maskf)
    logeuclid_distance_image = sitk.Multiply(logeuclid_distance_image,maskf)
    reimann_distance_image = sitk.Multiply(reimann_distance_image,maskf)
    kullback_distance_image = sitk.Multiply(kullback_distance_image,maskf)
    #
    ## clip outliers by computing 95 percentiles
    p = 95.0
    # FA
    fa_distance_arr = sitk.GetArrayFromImage(fa_distance_image)
    np.clip(fa_distance_arr, fa_distance_arr.min(), np.percentile(fa_distance_arr,p), fa_distance_arr)
    fa_distance_image = sitk.GetImageFromArray(fa_distance_arr)
    fa_distance_image.CopyInformation(mask)
    # MD
    md_distance_arr = sitk.GetArrayFromImage(md_distance_image)
    np.clip(md_distance_arr, md_distance_arr.min(), np.percentile(md_distance_arr,p), md_distance_arr)
    md_distance_image = sitk.GetImageFromArray(md_distance_arr)
    md_distance_image.CopyInformation(mask)
    # RD
    rd_distance_arr = sitk.GetArrayFromImage(rd_distance_image)
    np.clip(rd_distance_arr, rd_distance_arr.min(), np.percentile(rd_distance_arr,p), rd_distance_arr)
    rd_distance_image = sitk.GetImageFromArray(rd_distance_arr)
    rd_distance_image.CopyInformation(mask)
    # AD
    ad_distance_arr = sitk.GetArrayFromImage(ad_distance_image)
    np.clip(ad_distance_arr, ad_distance_arr.min(), np.percentile(ad_distance_arr,p), ad_distance_arr)
    ad_distance_image = sitk.GetImageFromArray(ad_distance_arr)
    ad_distance_image.CopyInformation(mask)
    # frobenius
    frobenius_distance_arr = sitk.GetArrayFromImage(frobenius_distance_image)
    np.clip(frobenius_distance_arr, frobenius_distance_arr.min(), np.percentile(frobenius_distance_arr,p), frobenius_distance_arr)
    frobenius_distance_image = sitk.GetImageFromArray(frobenius_distance_arr)
    frobenius_distance_image.CopyInformation(mask)
    # logeuclid
    logeuclid_distance_arr = sitk.GetArrayFromImage(logeuclid_distance_image)
    np.clip(logeuclid_distance_arr, logeuclid_distance_arr.min(), np.percentile(logeuclid_distance_arr,p), logeuclid_distance_arr)
    logeuclid_distance_image = sitk.GetImageFromArray(logeuclid_distance_arr)
    logeuclid_distance_image.CopyInformation(mask)
    # reimann
    reimann_distance_arr = sitk.GetArrayFromImage(reimann_distance_image)
    np.clip(reimann_distance_arr, reimann_distance_arr.min(), np.percentile(reimann_distance_arr,p), reimann_distance_arr)
    reimann_distance_image = sitk.GetImageFromArray(reimann_distance_arr)
    reimann_distance_image.CopyInformation(mask)
    # kullback
    kullback_distance_arr = sitk.GetArrayFromImage(kullback_distance_image)
    np.clip(kullback_distance_arr, kullback_distance_arr.min(), np.percentile(kullback_distance_arr,p), kullback_distance_arr)
    kullback_distance_image = sitk.GetImageFromArray(kullback_distance_arr)
    kullback_distance_image.CopyInformation(mask)
    #
    ## Write out the distance images
    sitk.WriteImage(fa_distance_image,fa_distance_image_fn)
    sitk.WriteImage(md_distance_image,md_distance_image_fn)
    sitk.WriteImage(rd_distance_image,rd_distance_image_fn)
    sitk.WriteImage(ad_distance_image,ad_distance_image_fn)
    sitk.WriteImage(frobenius_distance_image,frobenius_distance_image_fn)
    sitk.WriteImage(logeuclid_distance_image,logeuclid_distance_image_fn)
    sitk.WriteImage(reimann_distance_image,reimann_distance_image_fn)
    sitk.WriteImage(kullback_distance_image,kullback_distance_image_fn)
    #
    return [fa_distance_image_fn,
            md_distance_image_fn,
            rd_distance_image_fn,
            ad_distance_image_fn,
            frobenius_distance_image_fn,
            logeuclid_distance_image_fn,
            reimann_distance_image_fn,
            kullback_distance_image_fn]

def MakeInputSRList(DWI_SR_NN, DWI_SR_IFFT, DWI_SR_TV, DWI_SR_WTV):
    imagesList = [DWI_SR_NN, DWI_SR_IFFT, DWI_SR_TV, DWI_SR_WTV]
    return imagesList

def MakePurePlugsMaskInputList(inputT1, inputT2, inputIDWI):
    imagesList = [inputT1, inputT2, inputIDWI]
    return imagesList

def CreateBrainPurePlugsMask(PurePlugsMask, DWI_brainMask):
    import SimpleITK as sitk
    import os
    assert os.path.exists(PurePlugsMask), "File not found: %s" % PurePlugsMask
    assert os.path.exists(DWI_brainMask), "File not found: %s" % DWI_brainMask
    ppmask = sitk.ReadImage(PurePlugsMask)
    brainmask = sitk.ReadImage(DWI_brainMask) 
    brainPurePlugs = ppmask * sitk.Cast(brainmask,sitk.sitkUInt8)
    BrainPurePlugsMask = os.path.realpath('BrainPurePlugsMask.nrrd')
    sitk.WriteImage(brainPurePlugs,BrainPurePlugsMask)
    assert os.path.exists(BrainPurePlugsMask), "Output BrainPurePlugsMask file not found: %s" % BrainPurePlugsMask
    return BrainPurePlugsMask

def CreateWMRegions(LobesLabelMapVolume,BrainPurePlugsMask):
    import SimpleITK as sitk
    import os
    assert os.path.exists(LobesLabelMapVolume), "File not found: %s" % LobesLabelMapVolume
    assert os.path.exists(BrainPurePlugsMask), "File not found: %s" % BrainPurePlugsMask
    # read in masks
    lobeLabels = sitk.ReadImage(LobesLabelMapVolume)
    ppmask = sitk.ReadImage(BrainPurePlugsMask)
    # define lobe labels
    temporal_wm = ((lobeLabels == 9703) + (lobeLabels == 9803))
    occipital_wm = ((lobeLabels == 9706) + (lobeLabels == 9806))
    frontal_wm = ((lobeLabels == 9707) + (lobeLabels == 9807))
    parietal_wm = ((lobeLabels == 9708) + (lobeLabels == 9808))
    # resample each lobe label to lattice space of DWI 
    # (use: sitk.linear interpolation + thresholding at 0.49)
    resFilt = sitk.ResampleImageFilter()
    resFilt.SetReferenceImage(ppmask)
    resFilt.SetOutputPixelType(sitk.sitkFloat32)
    resFilt.SetInterpolator(sitk.sitkLinear)
    temporal_wm = resFilt.Execute(temporal_wm)
    # Thresholding by 0.49
    temporal_wm = sitk.BinaryThreshold(temporal_wm,0.49,1.0,1,0)
    #
    resFilt = sitk.ResampleImageFilter()
    resFilt.SetReferenceImage(ppmask)
    resFilt.SetOutputPixelType(sitk.sitkFloat32)
    resFilt.SetInterpolator(sitk.sitkLinear)
    occipital_wm = resFilt.Execute(occipital_wm)
    occipital_wm = sitk.BinaryThreshold(occipital_wm,0.49,1.0,1,0)
    #
    resFilt = sitk.ResampleImageFilter()
    resFilt.SetReferenceImage(ppmask)
    resFilt.SetOutputPixelType(sitk.sitkFloat32)
    resFilt.SetInterpolator(sitk.sitkLinear)
    frontal_wm = resFilt.Execute(frontal_wm)
    frontal_wm = sitk.BinaryThreshold(frontal_wm,0.49,1.0,1,0)
    #
    resFilt = sitk.ResampleImageFilter()
    resFilt.SetReferenceImage(ppmask)
    resFilt.SetOutputPixelType(sitk.sitkFloat32)
    resFilt.SetInterpolator(sitk.sitkLinear)
    parietal_wm = resFilt.Execute(parietal_wm)
    parietal_wm = sitk.BinaryThreshold(parietal_wm,0.49,1.0,1,0)
    # define pure/ not pure labels
    temporal_wm_pure = temporal_wm * ppmask
    temporal_wm_NOTpure = temporal_wm * (1 - ppmask)
    #
    occipital_wm_pure = occipital_wm * ppmask
    occipital_wm_NOTpure = occipital_wm * (1 - ppmask)
    #
    frontal_wm_pure = frontal_wm * ppmask
    frontal_wm_NOTpure = frontal_wm * (1 - ppmask)
    #
    parietal_wm_pure = frontal_wm * ppmask
    parietal_wm_NOTpure = frontal_wm * (1 - ppmask)
    # write out all 8 ROIs to the disk
    temporal_wm_pure_mask = os.path.realpath('temporal_wm_pure_mask.nrrd')
    sitk.WriteImage(temporal_wm_pure, temporal_wm_pure_mask)
    temporal_wm_NOTpure_mask = os.path.realpath('temporal_wm_NOTpure_mask.nrrd')
    sitk.WriteImage(temporal_wm_NOTpure, temporal_wm_NOTpure_mask)
    assert os.path.exists(temporal_wm_pure_mask), "Output temporal_wm_pure_mask file not found: %s" % temporal_wm_pure_mask
    assert os.path.exists(temporal_wm_NOTpure_mask), "Output temporal_wm_NOTpure_mask file not found: %s" % temporal_wm_NOTpure_mask
    #
    occipital_wm_pure_mask = os.path.realpath('occipital_wm_pure_mask.nrrd')
    sitk.WriteImage(occipital_wm_pure, occipital_wm_pure_mask)
    occipital_wm_NOTpure_mask = os.path.realpath('occipital_wm_NOTpure_mask.nrrd')
    sitk.WriteImage(occipital_wm_NOTpure, occipital_wm_NOTpure_mask)
    assert os.path.exists(occipital_wm_pure_mask), "Output occipital_wm_pure_mask file not found: %s" % occipital_wm_pure_mask
    assert os.path.exists(occipital_wm_NOTpure_mask), "Output occipital_wm_NOTpure_mask file not found: %s" % occipital_wm_NOTpure_mask
    #
    frontal_wm_pure_mask = os.path.realpath('frontal_wm_pure_mask.nrrd')
    sitk.WriteImage(frontal_wm_pure, frontal_wm_pure_mask)
    frontal_wm_NOTpure_mask = os.path.realpath('frontal_wm_NOTpure_mask.nrrd')
    sitk.WriteImage(frontal_wm_NOTpure, frontal_wm_NOTpure_mask)
    assert os.path.exists(frontal_wm_pure_mask), "Output frontal_wm_pure_mask file not found: %s" % frontal_wm_pure_mask
    assert os.path.exists(frontal_wm_NOTpure_mask), "Output frontal_wm_NOTpure_mask file not found: %s" % frontal_wm_NOTpure_mask
    #
    parietal_wm_pure_mask = os.path.realpath('parietal_wm_pure_mask.nrrd')
    sitk.WriteImage(parietal_wm_pure, parietal_wm_pure_mask)
    parietal_wm_NOTpure_mask = os.path.realpath('parietal_wm_NOTpure_mask.nrrd')
    sitk.WriteImage(parietal_wm_NOTpure, parietal_wm_NOTpure_mask)
    assert os.path.exists(parietal_wm_pure_mask), "Output parietal_wm_pure_mask file not found: %s" % parietal_wm_pure_mask
    assert os.path.exists(parietal_wm_NOTpure_mask), "Output parietal_wm_NOTpure_mask file not found: %s" % parietal_wm_NOTpure_mask
    #
    return [temporal_wm_pure_mask,temporal_wm_NOTpure_mask,
            occipital_wm_pure_mask,occipital_wm_NOTpure_mask,
            frontal_wm_pure_mask,frontal_wm_NOTpure_mask,
            parietal_wm_pure_mask,parietal_wm_NOTpure_mask]
    
    
    
#################################
#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\#

DistWF = pe.Workflow(name=WFname)

BASE_DIR = os.path.join('/scratch/TESTS/IpythonNotebook/20160615_HCPWF', '3_DistImagesWF')
DistWF.base_dir = BASE_DIR

inputsSpec = pe.Node(interface=IdentityInterface(fields=['inputT1',
                                                         'inputT2'
                                                         'LobesLabelMapVolume',
                                                         'DWI_brainMask',
                                                         'DWI_corrected_alignedSpace_masked',
                                                         'DWI_Baseline',
                                                         'DWI_SR_NN',
                                                         'DWI_SR_IFFT',
                                                         'DWI_SR_TV',
                                                         'DWI_SR_WTV'
                                                        ]),
                         name='inputsSpec')
inputsSpec.inputs.inputT1 = inputT1
inputsSpec.inputs.inputT2 = inputT2
inputsSpec.inputs.LobesLabelMapVolume = LobesLabelMapVolume
inputsSpec.inputs.DWI_corrected_alignedSpace_masked = DWI_corrected_alignedSpace_masked
inputsSpec.inputs.DWI_brainMask = DWI_brainMask
inputsSpec.inputs.DWI_Baseline = DWI_baseline
inputsSpec.inputs.DWI_NN = DWI_sr_nn
inputsSpec.inputs.DWI_IFFT = DWI_sr_ifft

outputsSpec = pe.Node(interface=IdentityInterface(fields=['FA_distance',
                                                          'MD_distance',
                                                          'RD_distance',
                                                          'AD_distance',
                                                          'Frobenius_distance',
                                                          'Logeuclid_distance',
                                                          'Reimann_distance',
                                                          'Kullback_distance',
                                                          'idwi_image',
                                                          'PurePlugsMask'
                                                         ]),
                      name='outputsSpec')


##
## Step 1: Create input list
##
MakeInputSRListNode = pe.Node(Function(function=MakeInputSRList,
                                       input_names=['DWI_SR_NN','DWI_SR_IFFT','DWI_SR_TV','DWI_SR_WTV'],
                                       output_names=['imagesList']),
                                       name="MakeInputSRList")
DistWF.connect([(inputsSpec,MakeInputSRListNode,[('DWI_SR_NN','DWI_SR_NN'),
                                                 ('DWI_SR_IFFT','DWI_SR_IFFT'),
                                                 ('DWI_SR_TV','DWI_SR_TV'),
                                                 ('DWI_SR_WTV','DWI_SR_WTV')
                                                ])])
##
## Step 2: Create distance images
##
ComputeDistanceImages = pe.MapNode(interface=Function(function = ComputeDistanceImages,
                                   input_names=['DWI_baseline','DWI_sr','DWI_brainMask'],
                                   output_names=['fa_distance_image_fn','md_distance_image_fn',
                                                 'rd_distance_image_fn','ad_distance_image_fn',
                                                 'frobenius_distance_image_fn','logeuclid_distance_image_fn',
                                                 'reimann_distance_image_fn','kullback_distance_image_fn'
                                                ]),
                                   name="ComputeDistanceImages",
                                   iterfield=['DWI_sr'])
DistWF.connect(MakeInputSRListNode, 'imagesList', ComputeDistanceImages, 'DWI_sr')
DistWF.connect([(inputsSpec,ComputeDistanceImages,[('DWI_Baseline','DWI_baseline'),
                                                   ('DWI_brainMask','DWI_brainMask')])])
DistWF.connect([(ComputeDistanceImages,outputsSpec,[('fa_distance_image_fn','FA_distance'),
                                                    ('md_distance_image_fn','MD_distance'),
                                                    ('rd_distance_image_fn','RD_distance'),
                                                    ('ad_distance_image_fn','AD_distance'),
                                                    ('frobenius_distance_image_fn','Frobenius_distance'),
                                                    ('logeuclid_distance_image_fn','Logeuclid_distance'),
                                                    ('reimann_distance_image_fn','Reimann_distance'),
                                                    ('kullback_distance_image_fn','Kullback_distance')])])

##
## Step 3: Create IDWI image
##
DTIEstim = pe.Node(interface=dtiestim(), name="DTIEstim")
DTIEstim.inputs.method = 'wls'
DTIEstim.inputs.threshold = 0
#DTIEstim.inputs.correctionType = 'nearest'
DTIEstim.inputs.tensor_output = 'DTI_Output.nrrd'
DTIEstim.inputs.idwi = 'IDWI_Output.nrrd'
DTIEstim.inputs.B0 = 'average_B0.nrrd'
DistWF.connect(inputsSpec, 'DWI_corrected_alignedSpace_masked', DTIEstim, 'dwi_image')
DistWF.connect(inputsSpec, 'DWI_brainMask', DTIEstim, 'brain_mask')
DistWF.connect(DTIEstim, 'idwi', outputsSpec, 'idwi_image')

##
## Step 4: Generate Pure plugs mask
##

## Step 4_1: Make a list for input modality list
MakePurePlugsMaskInputListNode = pe.Node(Function(function=MakePurePlugsMaskInputList,
                                                  input_names=['inputT1','inputT2','inputIDWI'],
                                                  output_names=['imagesList']),
                                         name="MakePurePlugsMaskInputList")
DistWF.connect(inputsSpec, 'inputT1', MakePurePlugsMaskInputListNode, 'inputT1')
DistWF.connect(inputsSpec, 'inputT2', MakePurePlugsMaskInputListNode, 'inputT2')
DistWF.connect(DTIEstim, 'idwi', MakePurePlugsMaskInputListNode, 'inputIDWI')

## Step 4_2: Create the mask
PurePlugsMaskNode = pe.Node(interface=GeneratePurePlugMask(), name="PurePlugsMask")
PurePlugsMaskNode.inputs.threshold = 0.15
PurePlugsMaskNode.inputs.outputMaskFile = 'PurePlugsMask.nrrd'
DistWF.connect(MakePurePlugsMaskInputListNode, 'imagesList', PurePlugsMaskNode, 'inputImageModalities')

## Step 4_3: 
CreateBrainPurePlugsMaskNode = pe.Node(Function(function=CreateBrainPurePlugsMask,
                                                input_names=['PurePlugsMask', 'DWI_brainMask'],
                                                output_names=['BrainPurePlugsMask']),
                                       name="CreateBrainPurePlugsMask")
DistWF.connect(PurePlugsMaskNode, 'outputMaskFile', CreateBrainPurePlugsMaskNode, 'PurePlugsMask')
DistWF.connect(inputsSpec, 'DWI_brainMask', CreateBrainPurePlugsMaskNode, 'DWI_brainMask')
DistWF.connect(CreateBrainPurePlugsMaskNode, 'BrainPurePlugsMask', outputsSpec, 'PurePlugsMask')


##
## Step 5: Generate different WM ROIs
##
CreateWMRegionsNode = pe.Node(Function(function=CreateWMRegions,
                                       input_names=['LobesLabelMapVolume','BrainPurePlugsMask'],
                                       output_names=['temporal_wm_pure_mask','temporal_wm_NOTpure_mask',
                                                     'occipital_wm_pure_mask','occipital_wm_NOTpure_mask',
                                                     'frontal_wm_pure_mask','frontal_wm_NOTpure_mask',
                                                     'parietal_wm_pure_mask','parietal_wm_NOTpure_mask']),
                                       name="CreateWMRegions")
DistWF.connect(inputsSpec, 'LobesLabelMapVolume', CreateWMRegionsNode, 'LobesLabelMapVolume')
DistWF.connect(CreateBrainPurePlugsMaskNode, 'BrainPurePlugsMask', outputsSpec, 'BrainPurePlugsMask')


##
## Step 6: Compute Statistics: compute average of each distance image in each of ROIs.
##
ComputeStats = pe.MapNode(interface=Function(function = ComputeStatistics,
                                             input_names=['temporal_wm_pure_mask','temporal_wm_NOTpure_mask',
                                                          'occipital_wm_pure_mask','occipital_wm_NOTpure_mask',
                                                          'frontal_wm_pure_mask','frontal_wm_NOTpure_mask',
                                                          'parietal_wm_pure_mask','parietal_wm_NOTpure_mask',
                                                          'FA_distance',
                                                          'MD_distance',
                                                          'RD_distance',
                                                          'AD_distance',
                                                          'Frobenius_distance',
                                                          'Logeuclid_distance',
                                                          'Reimann_distance',
                                                          'Kullback_distance'],
                                             output_names=[]),
                                    name="ComputeStatistics",
                                    iterfield=['FA_distance','MD_distance','RD_distance','AD_distance',
                                               'Frobenius_distance','Logeuclid_distance','Reimann_distance','Kullback_distance'])
DistWF.connect([(ComputeDistanceImages,ComputeStats,[('fa_distance_image_fn','FA_distance'),
                                                     ('md_distance_image_fn','MD_distance'),
                                                     ('rd_distance_image_fn','RD_distance'),
                                                     ('ad_distance_image_fn','AD_distance'),
                                                     ('frobenius_distance_image_fn','Frobenius_distance'),
                                                     ('logeuclid_distance_image_fn','Logeuclid_distance'),
                                                     ('reimann_distance_image_fn','Reimann_distance'),
                                                     ('kullback_distance_image_fn','Kullback_distance')])])





DWIDataSink = pe.Node(interface=nio.DataSink(), name='DWIDataSink')
DWIDataSink.overwrite = True
DWIDataSink.inputs.base_directory = BASE_DIR
DistWF.connect(outputsSpec, 'FA_distance', DWIDataSink, 'Outputs.@FA_distance')
DistWF.connect(outputsSpec, 'MD_distance', DWIDataSink, 'Outputs.@MD_distance')
DistWF.connect(outputsSpec, 'RD_distance', DWIDataSink, 'Outputs.@RD_distance')
DistWF.connect(outputsSpec, 'AD_distance', DWIDataSink, 'Outputs.@AD_distance')
DistWF.connect(outputsSpec, 'Frobenius_distance', DWIDataSink, 'Outputs.@Frobenius_distance')
DistWF.connect(outputsSpec, 'Logeuclid_distance', DWIDataSink, 'Outputs.@Logeuclid_distance')
DistWF.connect(outputsSpec, 'Reimann_distance', DWIDataSink, 'Outputs.@Reimann_distance')
DistWF.connect(outputsSpec, 'Kullback_distance', DWIDataSink, 'Outputs.@Kullback_distance')

DistWF.write_graph()
#DistWF.run()

NameError: name 'WFname' is not defined

In [ ]:
def ComputeStatistics(temporal_wm_pure_mask,temporal_wm_NOTpure_mask,
                      occipital_wm_pure_mask,occipital_wm_NOTpure_mask,
                      frontal_wm_pure_mask,frontal_wm_NOTpure_mask,
                      parietal_wm_pure_mask,parietal_wm_NOTpure_mask,
                      FA_distance,MD_distance,RD_distance,AD_distance,
                      Frobenius_distance,Logeuclid_distance,Reimann_distance,Kullback_distance):
    import os
    import SimpleITK as sitk
    ####
    def ReturnROIMean(distImage_fn, mask_fn):
        roi = sitk.ReadImage(mask_fn)
        distImage = sitk.ReadImage(distImage_fn)        
        statFilter = sitk.LabelStatisticsImageFilter()
        statFilter.Execute(distImage, roi)
        return statFilter.GetMean(1)
    ####
    assert os.path.exists(temporal_wm_pure_mask), "File not found: %s" % temporal_wm_pure_mask
    assert os.path.exists(temporal_wm_NOTpure_mask), "File not found: %s" % temporal_wm_NOTpure_mask
    assert os.path.exists(occipital_wm_pure_mask), "File not found: %s" % occipital_wm_pure_mask
    assert os.path.exists(occipital_wm_NOTpure_mask), "File not found: %s" % occipital_wm_NOTpure_mask
    assert os.path.exists(frontal_wm_pure_mask), "File not found: %s" % frontal_wm_pure_mask
    assert os.path.exists(frontal_wm_NOTpure_mask), "File not found: %s" % frontal_wm_NOTpure_mask
    assert os.path.exists(parietal_wm_pure_mask), "File not found: %s" % parietal_wm_pure_mask
    assert os.path.exists(parietal_wm_NOTpure_mask), "File not found: %s" % parietal_wm_NOTpure_mask
    assert os.path.exists(FA_distance), "File not found: %s" % FA_distance
    assert os.path.exists(MD_distance), "File not found: %s" % MD_distance
    assert os.path.exists(RD_distance), "File not found: %s" % RD_distance
    assert os.path.exists(AD_distance), "File not found: %s" % AD_distance
    assert os.path.exists(Frobenius_distance), "File not found: %s" % Frobenius_distance
    assert os.path.exists(Logeuclid_distance), "File not found: %s" % Logeuclid_distance
    assert os.path.exists(Reimann_distance), "File not found: %s" % Reimann_distance
    assert os.path.exists(Kullback_distance), "File not found: %s" % Kullback_distance
#     # read in mask images
#     temporal_wm_pure = sitk.ReadImage(temporal_wm_pure_mask)
#     temporal_wm_NOTpure = sitk.ReadImage(temporal_wm_NOTpure_mask)
#     occipital_wm_pure = sitk.ReadImage(occipital_wm_pure_mask)
#     occipital_wm_NOTpure = sitk.ReadImage(occipital_wm_NOTpure_mask)
#     frontal_wm_pure = sitk.ReadImage(frontal_wm_pure_mask)
#     frontal_wm_NOTpure = sitk.ReadImage(frontal_wm_NOTpure_mask)
#     parietal_wm_pure = sitk.ReadImage(parietal_wm_pure_mask)
#     parietal_wm_NOTpure = sitk.ReadImage(parietal_wm_NOTpure_mask)
#     # read in error images
#     fa_dist = sitk.ReadImage(FA_distance)
#     md_dist = sitk.ReadImage(MD_distance)
#     rd_dist = sitk.ReadImage(RD_distance)
#     ad_dist = sitk.ReadImage(AD_distance)
#     fro_dist = sitk.ReadImage(Frobenius_distance)
#     logeuc_dist = sitk.ReadImage(Logeuclid_distance)
#     reimann_dist = sitk.ReadImage(Reimann_distance)
#     kullback_dist = sitk.ReadImage(Kullback_distance)
    fa_stats_list = [ReturnROIMean(FA_distance,temporal_wm_pure_mask),
                     ReturnROIMean(FA_distance,occipital_wm_pure_mask),
                     ReturnROIMean(FA_distance,frontal_wm_pure_mask),
                     ReturnROIMean(FA_distance,parietal_wm_pure_mask),
                     ReturnROIMean(FA_distance,temporal_wm_NOTpure_mask),
                     ReturnROIMean(FA_distance,occipital_wm_NOTpure_mask),
                     ReturnROIMean(FA_distance,frontal_wm_NOTpure_mask),
                     ReturnROIMean(FA_distance,parietal_wm_NOTpure_mask)]
    #
    md_stats_list = [ReturnROIMean(MD_distance,temporal_wm_pure_mask),
                     ReturnROIMean(MD_distance,occipital_wm_pure_mask),
                     ReturnROIMean(MD_distance,frontal_wm_pure_mask),
                     ReturnROIMean(MD_distance,parietal_wm_pure_mask),
                     ReturnROIMean(MD_distance,temporal_wm_NOTpure_mask),
                     ReturnROIMean(MD_distance,occipital_wm_NOTpure_mask),
                     ReturnROIMean(MD_distance,frontal_wm_NOTpure_mask),
                     ReturnROIMean(MD_distance,parietal_wm_NOTpure_mask)]
    #
    rd_stats_list = [ReturnROIMean(RD_distance,temporal_wm_pure_mask),
                     ReturnROIMean(RD_distance,occipital_wm_pure_mask),
                     ReturnROIMean(RD_distance,frontal_wm_pure_mask),
                     ReturnROIMean(RD_distance,parietal_wm_pure_mask),
                     ReturnROIMean(RD_distance,temporal_wm_NOTpure_mask),
                     ReturnROIMean(RD_distance,occipital_wm_NOTpure_mask),
                     ReturnROIMean(RD_distance,frontal_wm_NOTpure_mask),
                     ReturnROIMean(RD_distance,parietal_wm_NOTpure_mask)]
    #
    md_stats_list = [ReturnROIMean(MD_distance,temporal_wm_pure_mask),
                     ReturnROIMean(MD_distance,occipital_wm_pure_mask),
                     ReturnROIMean(MD_distance,frontal_wm_pure_mask),
                     ReturnROIMean(MD_distance,parietal_wm_pure_mask),
                     ReturnROIMean(MD_distance,temporal_wm_NOTpure_mask),
                     ReturnROIMean(MD_distance,occipital_wm_NOTpure_mask),
                     ReturnROIMean(MD_distance,frontal_wm_NOTpure_mask),
                     ReturnROIMean(MD_distance,parietal_wm_NOTpure_mask)]

In [13]:
fa_dist_img = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/DistanceImagesWorkflow_CACHE_105115/ComputeDistanceImages/mapflow/_ComputeDistanceImages0/NN_FA_distance.nrrd'

In [14]:
print(fa_dist_img)

/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/DistanceImagesWorkflow_CACHE_105115/ComputeDistanceImages/mapflow/_ComputeDistanceImages0/NN_FA_distance.nrrd


In [15]:
dist_img_file_name = os.path.basename(fa_dist_img)
print(dist_img_file_name)
dist_img_base = os.path.splitext(dist_img_file_name)[0]
print(dist_img_base)
sr_name = dist_img_base.split('_',2)[0]
print(sr_name)
fa_distance_image_fn = os.path.realpath(sr_name + '_errorImages_statistics.nrrd')
print(fa_distance_image_fn)

NN_FA_distance.nrrd
NN_FA_distance
NN
/Volumes/scratch/Ipython_conda/SimpleITK-Notebook-Answers/NN_errorImages_statistics.nrrd


In [29]:
DWI_sr = '/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/SuperResolutionWorkflow_CACHE_105115/runSR/DWI_SR_IFFT.nrrd'

In [30]:
print(DWI_sr)

/Volumes/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/SuperResolutionWorkflow_CACHE_105115/runSR/DWI_SR_IFFT.nrrd


In [31]:
sr_file_name = os.path.basename(DWI_sr)
print(sr_file_name)
sr_file_name_base = os.path.splitext(sr_file_name)[0]
print(sr_file_name_base)
sr_name = sr_file_name_base.split('_',2)[2]
print(sr_name)
fa_distance_image_fn = os.path.realpath(sr_name + '_FA_distance.nrrd')
print(fa_distance_image_fn)

DWI_SR_IFFT.nrrd
DWI_SR_IFFT
IFFT
/Volumes/scratch/Ipython_conda/SimpleITK-Notebook-Answers/IFFT_FA_distance.nrrd


In [ ]:
def ComputeDistanceImages(DWI_baseline,DWI_sr,DWI_brainMask):
    import os
    import numpy as np
    from scipy.linalg import logm
    from scipy.linalg import eigvalsh # eigvalsh(A,B) is joint eigenvalues of A and B
    import SimpleITK as sitk
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
    #########
    ### distance functions ###
    # Frobenius distance
    def distance_euclid(tenfit_A, tenfit_B):
        return np.linalg.norm(tenfit_A.quadratic_form - tenfit_B.quadratic_form, ord='fro')
    # Log-Euclidian distance
    def distance_logeuclid(tenfit_A, tenfit_B):
        def fro_norm(A,B):
            return np.linalg.norm(A-B, ord='fro')
        return fro_norm(logm(tenfit_A.quadratic_form),logm(tenfit_B.quadratic_form))
    # Reimannian distance
    def distance_reimann(tenfit_A, tenfit_B):
        return np.sqrt((np.log(eigvalsh(tenfit_A.quadratic_form,tenfit_B.quadratic_form))**2).sum())
    # Kullback-Leibler distance
    def distance_kullback(tenfit_A, tenfit_B):
        A = tenfit_A.quadratic_form
        B = tenfit_B.quadratic_form
        dim = A.shape[0]
        kl = np.sqrt( np.trace( np.dot(np.linalg.inv(A),B)+np.dot(np.linalg.inv(B),A) ) - 2*dim )
        return 0.5*kl
    ######### 
    assert os.path.exists(DWI_baseline), "File not found: %s" % DWI_baseline
    assert os.path.exists(DWI_sr), "File not found: %s" % DWI_sr
    assert os.path.exists(DWI_brainMask), "File not found: %s" % DWI_brainMask
    mask = sitk.ReadImage(DWI_brainMask)
    # read DWI nrrd files
    data_base,nrrd_header_base,bvecs_base,bvals_base,grad_idx_base = ReadNAMICDWIFromNrrd(DWI_baseline)
    data_sr,nrrd_header_sr,bvecs_sr,bvals_sr,grad_idx_sr = ReadNAMICDWIFromNrrd(DWI_sr)
    # compute tensorfits
    from dipy.core.gradients import gradient_table
    from dipy.reconst.dti import TensorModel
    # base tensorfit
    gtab_base = gradient_table(bvals_base, bvecs_base)
    ten_base = TensorModel(gtab_base)
    tenfit_base = ten_base.fit(data_base)
    # super-res reconstructed tensorfit
    gtab_sr = gradient_table(bvals_sr, bvecs_sr)
    ten_sr = TensorModel(gtab_sr)
    tenfit_sr = ten_base.fit(data_sr)
    # create empty distance images
    size = [data_base.shape[0],data_base.shape[1],data_base.shape[2]] #data_base.shape[4] is gradient components
    fa_distance_image = sitk.Image(size, sitk.sitkFloat32)
    fa_distance_image.CopyInformation(mask)
    #
    md_distance_image = sitk.Image(size, sitk.sitkFloat32)
    md_distance_image.CopyInformation(mask)
    #
    rd_distance_image = sitk.Image(size, sitk.sitkFloat32)
    rd_distance_image.CopyInformation(mask)
    #
    ad_distance_image = sitk.Image(size, sitk.sitkFloat32)
    ad_distance_image.CopyInformation(mask)
    #
    frobenius_distance_image = sitk.Image(size, sitk.sitkFloat32)
    frobenius_distance_image.CopyInformation(mask)
    #
    logeuclid_distance_image = sitk.Image(size, sitk.sitkFloat32)
    logeuclid_distance_image.CopyInformation(mask)
    #
    reimann_distance_image = sitk.Image(size, sitk.sitkFloat32)
    reimann_distance_image.CopyInformation(mask)
    #
    kullback_distance_image = sitk.Image(size, sitk.sitkFloat32)
    kullback_distance_image.CopyInformation(mask)
    #
    # for loop to fill the distance images
    '''
    for i in xrange(size[0]):
        for j in xrange(size[1]):
            for k in xrange(size[2]):
                fa_distance_image[i,j,k] = abs(tenfit_base[i,j,k].fa - tenfit_sr[i,j,k].fa)
                md_distance_image[i,j,k] = abs(tenfit_base[i,j,k].md - tenfit_sr[i,j,k].md)
                rd_distance_image[i,j,k] = abs(tenfit_base[i,j,k].rd - tenfit_sr[i,j,k].rd)
                ad_distance_image[i,j,k] = abs(tenfit_base[i,j,k].ad - tenfit_sr[i,j,k].ad)
                frobenius_distance_image[i,j,k] = distance_euclid(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                logeuclid_distance_image[i,j,k] = distance_logeuclid(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                reimann_distance_image[i,j,k] = distance_reimann(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                kullback_distance_image[i,j,k] = distance_kullback(tenfit_base[i,j,k], tenfit_sr[i,j,k])
    '''
    #
    outDir = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/3_DistImagesWF'
    maskf = sitk.Cast(mask, sitk.sitkFloat32)
    #
    fa_distance_image = sitk.Multiply(fa_distance_image,maskf)
    fa_distance_image_fn = os.path.join(outDir, 'FA_distance.nrrd')
    sitk.WriteImage(fa_distance_image,fa_distance_image_fn)
    #
    md_distance_image = sitk.Multiply(md_distance_image,maskf)
    md_distance_image_fn = os.path.join(outDir, 'MD_distance.nrrd')
    sitk.WriteImage(md_distance_image,md_distance_image_fn)
    #
    rd_distance_image = sitk.Multiply(rd_distance_image,maskf)
    rd_distance_image_fn = os.path.join(outDir, 'RD_distance.nrrd')
    sitk.WriteImage(rd_distance_image,rd_distance_image_fn)
    #
    ad_distance_image = sitk.Multiply(ad_distance_image,maskf)
    ad_distance_image_fn = os.path.join(outDir, 'AD_distance.nrrd')
    sitk.WriteImage(ad_distance_image,ad_distance_image_fn)
    #
    frobenius_distance_image = sitk.Multiply(frobenius_distance_image,maskf)
    frobenius_distance_image_fn = os.path.join(outDir, 'Frobenius_distance.nrrd')
    sitk.WriteImage(frobenius_distance_image,frobenius_distance_image_fn)
    #
    logeuclid_distance_image = sitk.Multiply(logeuclid_distance_image,maskf)
    logeuclid_distance_image_fn = os.path.join(outDir, 'Logeuclid_distance.nrrd')
    sitk.WriteImage(logeuclid_distance_image,logeuclid_distance_image_fn)
    #
    reimann_distance_image = sitk.Multiply(reimann_distance_image,maskf)
    reimann_distance_image_fn = os.path.join(outDir, 'Reimann_distance.nrrd')
    sitk.WriteImage(reimann_distance_image,reimann_distance_image_fn)
    #
    kullback_distance_image = sitk.Multiply(kullback_distance_image,maskf)
    kullback_distance_image_fn = os.path.join(outDir, 'Kullback_distance.nrrd')
    sitk.WriteImage(kullback_distance_image,kullback_distance_image_fn)
    #
    return [fa_distance_image_fn,
            md_distance_image_fn,
            rd_distance_image_fn,
            ad_distance_image_fn,
            frobenius_distance_image_fn,
            logeuclid_distance_image_fn,
            reimann_distance_image_fn,
            kullback_distance_image_fn]

In [ ]:
[fa,md,rd,ad,fro,log,rei,kull] = ComputeDistanceImages(DWI_baseline,DWI_sr_ifft,DWI_brainMask)

In [ ]:
print(fa)

In [ ]:
def ComputeDistanceImages(DWI_baseline,DWI_sr,DWI_brainMask):
    import os
    import numpy as np
    from scipy.linalg import logm
    from scipy.linalg import eigvalsh # eigvalsh(A,B) is joint eigenvalues of A and B
    import SimpleITK as sitk
    from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
    #########
    ### distance functions ###
    # Frobenius distance
    def distance_euclid(tenfit_A, tenfit_B):
        return np.linalg.norm(tenfit_A.quadratic_form - tenfit_B.quadratic_form, ord='fro')
    # Log-Euclidian distance
    def distance_logeuclid(tenfit_A, tenfit_B):
        def fro_norm(A,B):
            return np.linalg.norm(A-B, ord='fro')
        return fro_norm(logm(tenfit_A.quadratic_form),logm(tenfit_B.quadratic_form))
    # Reimannian distance
    def distance_reimann(tenfit_A, tenfit_B):
        return np.sqrt((np.log(eigvalsh(tenfit_A.quadratic_form,tenfit_B.quadratic_form))**2).sum())
    # Kullback-Leibler distance
    def distance_kullback(tenfit_A, tenfit_B):
        A = tenfit_A.quadratic_form
        B = tenfit_B.quadratic_form
        dim = A.shape[0]
        kl = np.sqrt( np.trace( np.dot(np.linalg.inv(A),B)+np.dot(np.linalg.inv(B),A) ) - 2*dim )
        return 0.5*kl
    ######### 
    assert os.path.exists(DWI_baseline), "File not found: %s" % DWI_baseline
    assert os.path.exists(DWI_sr), "File not found: %s" % DWI_sr
    assert os.path.exists(DWI_brainMask), "File not found: %s" % DWI_brainMask
    mask = sitk.ReadImage(DWI_brainMask)
    # read DWI nrrd files
    data_base,nrrd_header_base,bvecs_base,bvals_base,grad_idx_base = ReadNAMICDWIFromNrrd(DWI_baseline)
    data_sr,nrrd_header_sr,bvecs_sr,bvals_sr,grad_idx_sr = ReadNAMICDWIFromNrrd(DWI_sr)
    # compute tensorfits
    from dipy.core.gradients import gradient_table
    from dipy.reconst.dti import TensorModel
    # base tensorfit
    gtab_base = gradient_table(bvals_base, bvecs_base)
    ten_base = TensorModel(gtab_base)
    tenfit_base = ten_base.fit(data_base)
    # super-res reconstructed tensorfit
    gtab_sr = gradient_table(bvals_sr, bvecs_sr)
    ten_sr = TensorModel(gtab_sr)
    tenfit_sr = ten_base.fit(data_sr)
    # create empty distance images
    size = [data_base.shape[0],data_base.shape[1],data_base.shape[2]] #data_base.shape[4] is gradient components
    fa_distance_image = sitk.Image(size, sitk.sitkFloat32)
    fa_distance_image.CopyInformation(mask)
    #
    # for loop to fill the distance images
    for i in xrange(size[0]):
        for j in xrange(size[1]):
            for k in xrange(size[2]):
                fa_distance_image[i,j,k] = abs(tenfit_base[i,j,k].fa - tenfit_sr[i,j,k].fa)
                #md_distance_image[i,j,k] = abs(tenfit_base[i,j,k].md - tenfit_sr[i,j,k].md)
                #rd_distance_image[i,j,k] = abs(tenfit_base[i,j,k].rd - tenfit_sr[i,j,k].rd)
                #ad_distance_image[i,j,k] = abs(tenfit_base[i,j,k].ad - tenfit_sr[i,j,k].ad)
                #frobenius_distance_image[i,j,k] = distance_euclid(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                #logeuclid_distance_image[i,j,k] = distance_logeuclid(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                #reimann_distance_image[i,j,k] = distance_reimann(tenfit_base[i,j,k], tenfit_sr[i,j,k])
                #kullback_distance_image[i,j,k] = distance_kullback(tenfit_base[i,j,k], tenfit_sr[i,j,k])
    #
    BASE_DIR = os.path.join('/scratch/TESTS/IpythonNotebook/20160615_HCPWF', '3_DistImagesWF')
    fa_distance_image_fn = os.path.join(BASE_DIR, 'FA_distance.nrrd')
    sitk.WriteImage(fa_distance_image,fa_distance_image_fn)
    #
    return fa_distance_image_fn

In [ ]:
fn = ComputeDistanceImages(DWI_baseline,DWI_sr_ifft,DWI_brainMask)

In [ ]:
print(fn)

In [ ]:
from ReadWriteNrrdDWI import ReadNAMICDWIFromNrrd, WriteNAMICDWIToNrrd
data_base,nrrd_header_base,bvecs_base,bvals_base,grad_idx_base = ReadNAMICDWIFromNrrd(DWI_baseline)

In [ ]:
data_base.shape

In [ ]:
mask = sitk.ReadImage(DWI_brainMask)
size=[data_base.shape[0],data_base.shape[1],data_base.shape[2]]
fa_distance_image = sitk.Image(size, sitk.sitkFloat32)
fa_distance_image.CopyInformation(mask)

In [ ]:
myshow(fa_distance_image)

In [ ]:
print(size[0])

In [ ]:
# compute tensorfits
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
# base tensorfit
gtab_base = gradient_table(bvals_base, bvecs_base)
ten_base = TensorModel(gtab_base)
tenfit_base = ten_base.fit(data_base)

In [ ]:
print(tenfit_base[70,80,70].fa)

In [ ]:
print(bvals_base)